<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/parity_classification_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane numpy scikit-learn

In [ ]:
#!/usr/bin/env python3

import itertools
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def generate_parity_data(n_bits=4):
    """
    Returns:
      X: array of shape (2^n_bits, n_bits) with all bitstrings
      y: array of shape (2^n_bits,) with parity labels {0,1}
    """
    X = np.array(list(itertools.product([0, 1], repeat=n_bits)))
    y = X.sum(axis=1) % 2
    return X, y

def analytic_cnot_cascade(X, n_bits=4):
    """
    Exact parity by cascading CNOTs onto an ancilla qubit.
    Returns predicted bits {0,1}.
    """
    dev = qml.device("default.qubit", wires=n_bits + 1)

    @qml.qnode(dev)
    def circuit(x):
        # Encode each input bit: apply X if bit == 1
        for i, bit in enumerate(x):
            if bit:
                qml.PauliX(wires=i)
        # Cascade CNOTs onto ancilla at wire n_bits
        for i in range(n_bits):
            qml.CNOT(wires=[i, n_bits])
        # Measure ancilla expectation of Z → +1 (even) or -1 (odd)
        return qml.expval(qml.PauliZ(wires=n_bits))

    preds = []
    for x in X:
        exp = circuit(x)
        # exp = +1 → parity even → label 0
        # exp = -1 → parity odd  → label 1
        preds.append(0 if exp > 0 else 1)

    return np.array(preds)

def variational_qnn_cv(X, y, n_bits=4, epochs=30, lr=0.4):
    """
    Variational QNN trained with mean squared error and
    evaluated via 5-fold cross-validation.
    Returns (mean_accuracy, std_accuracy).
    """
    dev = qml.device("default.qubit", wires=n_bits)

    @qml.qnode(dev, interface="autograd")
    def circuit(params, x):
        # Feature encoding: RY(pi) on qubit i if x[i] == 1
        for i, bit in enumerate(x):
            if bit:
                qml.RY(np.pi, wires=i)
        # Trainable rotations
        for i in range(n_bits):
            qml.RY(params[i], wires=i)
        # Entangling ladder
        for i in range(n_bits - 1):
            qml.CNOT(wires=[i, i + 1])
        # Readout on last qubit
        return qml.expval(qml.PauliZ(wires=n_bits - 1))

    # Map labels {0,1} → {+1,-1} for regression loss
    y_signed = 1 - 2 * y

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, test_idx in kf.split(X, y):
        # Initialize parameters
        params = pnp.random.randn(n_bits, requires_grad=True) * 0.1
        opt = qml.GradientDescentOptimizer(stepsize=lr)

        # Training loop
        for _ in range(epochs):
            def cost(p):
                preds = [circuit(p, X[i]) for i in train_idx]
                return pnp.mean((pnp.array(preds) - y_signed[train_idx]) ** 2)
            params = opt.step(cost, params)

        # Evaluation: sign of expectation → predicted bit
        preds_test = [circuit(params, X[i]) for i in test_idx]
        bits     = [(0 if p > 0 else 1) for p in preds_test]
        scores.append(accuracy_score(y[test_idx], bits))

    return np.mean(scores), np.std(scores)

def quantum_kernel_svm_cv(X, y, n_bits=4):
    """
    Constructs a quantum feature map, builds the precomputed kernel
    matrix, and evaluates an SVM with 5-fold cross-validation.
    Returns (mean_accuracy, std_accuracy).
    """
    dev = qml.device("default.qubit", wires=n_bits)

    @qml.qnode(dev)
    def feature_map(x, z=None):
        # Encoding x
        for i, bit in enumerate(x):
            if bit:
                qml.RY(np.pi, wires=i)
        for i in range(n_bits - 1):
            qml.CNOT(wires=[i, i + 1])
        # If z provided, apply the inverse encoding
        if z is not None:
            for i, bit in enumerate(z):
                if bit:
                    qml.RY(-np.pi, wires=i)
        return qml.state()

    # Precompute all statevectors
    states = [feature_map(x) for x in X]
    N = len(X)
    K = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            overlap = np.vdot(states[i], states[j])
            K[i, j] = np.abs(overlap) ** 2

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, test_idx in kf.split(K, y):
        svc = SVC(kernel="precomputed")
        K_train = K[np.ix_(train_idx, train_idx)]
        y_train = y[train_idx]
        svc.fit(K_train, y_train)

        K_test = K[np.ix_(test_idx, train_idx)]
        preds  = svc.predict(K_test)
        scores.append(accuracy_score(y[test_idx], preds))

    return np.mean(scores), np.std(scores)

if __name__ == "__main__":
    n_bits = 4
    X, y = generate_parity_data(n_bits=n_bits)

    # 1) Exact analytic circuit
    preds_exact = analytic_cnot_cascade(X, n_bits=n_bits)
    acc_exact   = accuracy_score(y, preds_exact)
    print(f"Analytic CNOT Cascade Accuracy: {acc_exact:.3f}")

    # 2) Variational QNN with 5-fold CV
    mean_var, std_var = variational_qnn_cv(X, y, n_bits=n_bits)
    print(f"Variational QNN CV Accuracy: {mean_var:.3f} ± {std_var:.3f}")

    # 3) Quantum Kernel + SVM with 5-fold CV
    mean_k, std_k = quantum_kernel_svm_cv(X, y, n_bits=n_bits)
    print(f"Quantum Kernel+SVM CV Accuracy: {mean_k:.3f} ± {std_k:.3f}")